In [8]:
import os
from dotenv import load_dotenv
from langsmith import Client, traceable

# 1. Load environment variables from .env
if load_dotenv(override=True):
    print("✅ .env file found and loaded.")
else:
    print("⚠️  No .env file found. Ensure your environment variables are set manually.")

# 2. Check for API Key
api_key = os.getenv("LANGCHAIN_API_KEY")
if not api_key:
    raise ValueError("❌ LANGCHAIN_API_KEY is missing. Please check your .env file.")

print(f"🔑 API Key detected: {api_key[:5]}...*****")

# 3. Define a simple traced function
@traceable(name="integration_test_run")
def hello_langsmith(name):
    return f"Hello, {name}! LangSmith is working."

def main():
    try:
        # Initialize Client to verify connection
        client = Client()
        
        # Send a test trace
        print("🚀 Sending test trace to LangSmith...")
        result = hello_langsmith("Developer")
        
        print(f"✅ Trace executed successfully. Result: '{result}'")
        print(f"👀 Check your project '{os.getenv('LANGCHAIN_PROJECT', 'default')}' in the LangSmith UI to see the trace.")
        
    except Exception as e:
        print(f"❌ Integration check failed: {e}")

if __name__ == "__main__":
    main()

✅ .env file found and loaded.
🔑 API Key detected: lsv2_...*****
🚀 Sending test trace to LangSmith...
✅ Trace executed successfully. Result: 'Hello, Developer! LangSmith is working.'
👀 Check your project 'aws' in the LangSmith UI to see the trace.


In [32]:
import os
import time
import uuid
from dotenv import load_dotenv
from langsmith import Client, traceable
from langsmith.utils import tracing_is_enabled

# 1. Load Envs
load_dotenv(override=True)

# 2. SETUP
# Note: LangSmith checks LANGCHAIN_PROJECT first, then LANGSMITH_PROJECT
target_project = os.getenv("LANGCHAIN_PROJECT") or os.getenv("LANGSMITH_PROJECT") or "default"
print(f"🎯 Target Project (Env): {target_project}")

client = Client()

# Generate a unique name for THIS specific execution
unique_run_name = f"test_run_{uuid.uuid4().hex[:8]}"
print(f"🆔 Generated Unique Run Name: {unique_run_name}")

# 3. DEFINE FUNCTION
@traceable(project_name=target_project)
def my_test_function(input_text):
    return f"Processed: {input_text}"

def verify_unique_run():
    print(f"🚀 Executing function with name '{unique_run_name}'...")
    
    try:
        # We pass 'langsmith_extra' to override the default name dynamically
        my_test_function(
            "Hello LangSmith", 
            langsmith_extra={"name": unique_run_name}
        )
        print("✅ Local execution finished.")
    except Exception as e:
        print(f"❌ Execution failed: {e}")
        return

    # 4. FLUSH / WAIT
    print("⏳ Flushing logs to LangSmith server...")
    try:
        client.flush()
    except:
        time.sleep(3) 

    # 5. VERIFY ON SERVER
    print("⏳ Waiting 5s for server indexing...")
    time.sleep(5)
    
    print(f"🔍 Searching server for run: '{unique_run_name}'...")
    
    # We list runs from the expected project to find our unique ID
    runs = list(client.list_runs(
        project_name=target_project,
        limit=5 
    ))

    found = False
    for run in runs:
        if run.name == unique_run_name:
            found = True
            
            # --- NEW LOGIC START ---
            # 1. Get the project/session ID from the run
            project_id = run.session_id
            
            # 2. Ask the client to fetch the full project details using that ID
            actual_project = client.read_project(project_id=project_id)
            # --- NEW LOGIC END ---

            print("\n" + "="*40)
            print("🎉 SUCCESS: INTEGRATION VERIFIED")
            print("="*40)
            print(f"📍 Run ID:             {run.id}")
            print(f"🏷️  Run Name:           {run.name}")
            print(f"🔗 Project (Expected): {target_project}")
            print(f"✅ Project (Client):   {actual_project.name}") # Verified from server
            print(f"⏱️  Latency:            {run.end_time - run.start_time}")
            break
            
    if not found:
        print("\n" + "="*40)
        print("❌ FAILED: Run not found on server")
        print("="*40)
        print(f"Expected Run Name: {unique_run_name}")
        print(f"Searching in Project: {target_project}")
        if runs:
            print(f"Latest run found in '{target_project}': '{runs[0].name}' (Mismatch)")
        else:
            print(f"No runs found at all in project '{target_project}'.")
            
if __name__ == "__main__":
    verify_unique_run()

🎯 Target Project (Env): aws
🆔 Generated Unique Run Name: test_run_9c26874a
🚀 Executing function with name 'test_run_9c26874a'...
✅ Local execution finished.
⏳ Flushing logs to LangSmith server...
⏳ Waiting 5s for server indexing...
🔍 Searching server for run: 'test_run_9c26874a'...

🎉 SUCCESS: INTEGRATION VERIFIED
📍 Run ID:             152b4d44-1e31-420d-a666-73545e48e711
🏷️  Run Name:           test_run_9c26874a
🔗 Project (Expected): aws
✅ Project (Client):   aws
⏱️  Latency:            0:00:00.000535


In [29]:
import os
import time
import uuid
from dotenv import load_dotenv
from langsmith import Client, traceable

# 1. FORCE RELOAD ENVS
if "LANGCHAIN_PROJECT" in os.environ:
    del os.environ["LANGCHAIN_PROJECT"]
if "LANGSMITH_PROJECT" in os.environ:
    del os.environ["LANGSMITH_PROJECT"]

load_dotenv(override=True)

# 2. DEFINE THE TARGET PROJECT
TARGET_PROJECT = "forced_debug_project"

# IMPORTANT FIX:
# We set the environment variable explicitly in Python.
# The Client automatically picks this up when it needs to create runs.
os.environ["LANGCHAIN_PROJECT"] = TARGET_PROJECT

print(f"\n--- 🚀 STARTING FORCED TEST: '{TARGET_PROJECT}' ---")

# Initialize Client (No arguments needed now)
client = Client()

unique_run_name = f"forced_run_{uuid.uuid4().hex[:8]}"

# We also enforce it in the decorator to be doubly sure
@traceable(project_name=TARGET_PROJECT)
def forced_function(x):
    return f"Processed: {x}"

def run_debug():
    print(f"Executing '{unique_run_name}'...")
    
    try:
        # Run the function
        forced_function("Force Check", langsmith_extra={"name": unique_run_name})
    except Exception as e:
        print(f"❌ Execution failed: {e}")
        return
    
    # Flush
    print("⏳ Flushing to server...")
    try:
        client.flush()
    except:
        time.sleep(3)
        
    time.sleep(5) 
    
    # Verify
    print(f"🔍 Verifying in project '{TARGET_PROJECT}'...")
    
    # List runs explicitly looking at the forced project name
    runs = list(client.list_runs(project_name=TARGET_PROJECT, limit=5))
    
    found = False
    for run in runs:
        if run.name == unique_run_name:
            found = True
            print("\n✅ SUCCESS: Found the run in the expected project!")
            print(f"   Run ID: {run.id}")
            # Correct way to get project name from run using the client
            try:
                project = client.read_project(project_id=run.session_id)
                print(f"   Project: {project.name}")
            except:
                print(f"   Project Session ID: {run.session_id}")
            break
            
    if not found:
        print(f"\n❌ FAILED: Still cannot find run in '{TARGET_PROJECT}'.")
        if runs:
            print(f"   Found other runs in this project: {[r.name for r in runs]}")
        else:
            print("   No runs found in this project at all.")

if __name__ == "__main__":
    run_debug()


--- 🚀 STARTING FORCED TEST: 'forced_debug_project' ---
Executing 'forced_run_ec722df0'...
⏳ Flushing to server...
🔍 Verifying in project 'forced_debug_project'...

✅ SUCCESS: Found the run in the expected project!
   Run ID: 8c646582-9bf1-4219-a2d3-27201fd8a67f
   Project: forced_debug_project
